# 14 — Sequential analysis: peeking vs valid monitoring

This notebook demonstrates why **naïve peeking** (re-checking p-values repeatedly) inflates type I error, and how **sequential monitoring** fixes it.

We cover (v1):
- Two-group A/B (control vs test)
- Mean metric (difference in means, user-level)
- Naïve repeated looks (invalid)
- Group sequential monitoring (valid for K pre-defined looks)

Outputs:
- Look table (per-look stats)
- z-trajectory vs boundaries
- Empirical type I error comparison via simulation

Notes:
- This implementation uses a normal approximation and Welch-style SE; interpret results accordingly.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.stats import norm

from tecore.sequential.schema import (
    SequentialSpec,
    LookSchedule,
    SequentialConfig,
    SequentialMode,
    SpendingFunction,
    EffectDirection,
)
from tecore.sequential.preprocess import build_look_table_mean
from tecore.sequential.group_sequential import run_group_sequential
from tecore.sequential.simulate import SequentialSimConfig, simulate_ab_stream

plt.rcParams["figure.figsize"] = (9, 5)
plt.rcParams["axes.grid"] = True

def fixed_p_from_z(z: float, two_sided: bool = True) -> float:
    if not np.isfinite(z):
        return np.nan
    return float(2 * norm.sf(abs(z))) if two_sided else float(norm.sf(z))

def run_naive_peeking_mean(df: pd.DataFrame, looks: list[int], alpha: float = 0.05) -> dict:
    """Naïve repeated testing: compute a standard fixed-horizon p-value at each look,
    and declare significance if ANY look has p < alpha.

    This inflates type I error.
    """
    spec = SequentialSpec(group_col="group", control_label="control", test_label="test", y_col="y", timestamp_col="timestamp")
    schedule = LookSchedule(looks=looks)
    cfg = SequentialConfig(
        mode=SequentialMode.GROUP_SEQUENTIAL,
        alpha=alpha,
        two_sided=True,
        spending=SpendingFunction.OBRIEN_FLEMING,
        effect_direction=EffectDirection.TWO_SIDED,
        min_n_per_group=10,
        var_floor=1e-12,
        seed=0,
    )
    lt, warn = build_look_table_mean(df, spec, schedule, cfg)
    z = pd.to_numeric(lt["z"], errors="coerce").to_numpy(dtype=float)
    p = np.array([fixed_p_from_z(zi, two_sided=True) for zi in z], dtype=float)
    stop_idx = int(np.where(p < alpha)[0][0]) if np.any(p < alpha) else None
    return {
        "look_table": lt.assign(p_fixed=p),
        "warnings": warn,
        "rejected": bool(stop_idx is not None),
        "stop_look": int(lt.iloc[stop_idx]["look_n"]) if stop_idx is not None else None,
        "min_p": float(np.nanmin(p)),
    }


## 1) Single run: how peeking can “find significance” under the null

We simulate a **null effect** stream (effect=0), check a sequence of looks, and compare:
- Naïve repeated testing (invalid)
- Group sequential O’Brien-Fleming (valid for the schedule)


In [ ]:
# Parameters for a small, fast demo
N = 5000
looks = [500, 1000, 1500, 2000, 3000, 4000, 5000]
alpha = 0.05

df = simulate_ab_stream(SequentialSimConfig(n=N, effect=0.0, noise_sd=1.0, heavy_tail=False, drift=False, seed=7, ratio=False))

# Naïve peeking
naive = run_naive_peeking_mean(df, looks=looks, alpha=alpha)
lt_naive = naive["look_table"].copy()

# Group sequential (OBF)
spec = SequentialSpec(group_col="group", control_label="control", test_label="test", y_col="y", timestamp_col="timestamp")
schedule = LookSchedule(looks=looks)
cfg_gs = SequentialConfig(
    mode=SequentialMode.GROUP_SEQUENTIAL,
    alpha=alpha,
    two_sided=True,
    spending=SpendingFunction.OBRIEN_FLEMING,
    effect_direction=EffectDirection.TWO_SIDED,
    min_n_per_group=10,
    var_floor=1e-12,
    seed=7,
)
lt, warn = build_look_table_mean(df, spec, schedule, cfg_gs)
res_gs = run_group_sequential(lt, cfg_gs)

display(lt_naive.head())
print("Naïve rejected?", naive["rejected"], "stop_look=", naive["stop_look"], "min_p=", naive["min_p"])
print("Group sequential decision:", res_gs.decision, "stopped=", res_gs.stopped, "stop_look=", res_gs.stop_look)


In [ ]:
## Plot: z-trajectory and fixed p-values vs alpha
x = lt_naive["look_n"].to_numpy()
z = lt_naive["z"].to_numpy(dtype=float)
p = lt_naive["p_fixed"].to_numpy(dtype=float)

fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.plot(x, z, label="z")
ax1.axhline(0.0, linewidth=1.0)
ax1.set_xlabel("look_n")
ax1.set_ylabel("z")
ax1.set_title("Null run: z trajectory across looks")
ax1.legend(loc="best")
plt.show()

fig = plt.figure()
ax2 = fig.add_subplot(111)
ax2.plot(x, p, marker="o", label="fixed p-value at look")
ax2.axhline(alpha, linestyle="--", label="alpha")
ax2.set_xlabel("look_n")
ax2.set_ylabel("p-value")
ax2.set_title("Null run: naïve peeking uses fixed p-values repeatedly")
ax2.set_ylim(0, 1)
ax2.legend(loc="best")
plt.show()


## 2) Simulation: type I error inflation from peeking

We estimate the empirical type I error under the null:
- Naïve peeking: reject if *any* look has fixed p < alpha
- Group sequential (OBF): reject if boundary is crossed

Expectation:
- Naïve peeking will typically exceed alpha
- Group sequential should be close to alpha (within Monte Carlo noise)


In [ ]:
n_sims = 300
N = 4000
looks = [400, 800, 1200, 1600, 2000, 3000, 4000]
alpha = 0.05

naive_reject = []
gs_reject = []
gs_stop = []
min_pvals = []

spec = SequentialSpec(group_col="group", control_label="control", test_label="test", y_col="y", timestamp_col="timestamp")
schedule = LookSchedule(looks=looks)
cfg_gs = SequentialConfig(
    mode=SequentialMode.GROUP_SEQUENTIAL,
    alpha=alpha,
    two_sided=True,
    spending=SpendingFunction.OBRIEN_FLEMING,
    effect_direction=EffectDirection.TWO_SIDED,
    min_n_per_group=10,
    var_floor=1e-12,
)

for s in range(n_sims):
    df = simulate_ab_stream(SequentialSimConfig(n=N, effect=0.0, noise_sd=1.0, heavy_tail=False, drift=False, seed=1000 + s, ratio=False))

    # naive peeking
    naive = run_naive_peeking_mean(df, looks=looks, alpha=alpha)
    naive_reject.append(naive["rejected"])
    min_pvals.append(naive["min_p"])

    # group sequential OBF
    lt, _ = build_look_table_mean(df, spec, schedule, cfg_gs)
    res = run_group_sequential(lt, cfg_gs)
    gs_reject.append(res.decision == "reject")
    gs_stop.append(res.stop_look if res.stop_look is not None else looks[-1])

naive_type1 = float(np.mean(naive_reject))
gs_type1 = float(np.mean(gs_reject))

print("Empirical type I error (naïve peeking):", naive_type1)
print("Empirical type I error (group sequential OBF):", gs_type1)

fig = plt.figure()
ax = fig.add_subplot(111)
ax.hist(min_pvals, bins=30)
ax.axvline(alpha, linestyle="--", label="alpha")
ax.set_title("Distribution of min fixed p-value across looks (null)")
ax.set_xlabel("min p-value over looks")
ax.set_ylabel("count")
ax.legend(loc="best")
plt.show()


## Interpretation

**What happened?**
- In naïve peeking, each look re-uses a fixed-horizon p-value without correcting for repeated testing. Under the null, this raises the probability of seeing a “lucky” small p-value at some look.
- In group sequential monitoring, we replace the fixed 1.96 threshold with a **sequential boundary** designed so the overall type I error across the pre-defined looks stays near α.

**Practical takeaway (v1):**
- If you want to look multiple times and keep a frequentist error guarantee, use a sequential procedure (e.g., OBF/Pocock) rather than raw p-values.

Limitations:
- This demo assumes i.i.d.-like observations; violations (drift, heavy tails) can degrade calibration and require guardrails.


## Summary (for article / report)

Fill in the placeholders after running:

- Schedule: `{looks}` (K = {K})
- α = {alpha}
- Empirical type I error under the null:
  - naïve peeking: **{naive_type1:.3f}**
  - group sequential (OBF): **{gs_type1:.3f}**

Claim:
- “Repeated looks with fixed p-values inflate false positives, while sequential boundaries maintain error control across looks.”


In [ ]:
## Commands to extract final numbers for the Summary section
K = len(looks)
print("looks=", looks)
print("K=", K)
print("alpha=", alpha)
print("naive_type1=", naive_type1)
print("gs_type1=", gs_type1)

# Optional: 95% MC error bars (Wald)
def mc_ci(p, n):
    se = np.sqrt(max(1e-12, p * (1 - p) / n))
    return (p - 1.96 * se, p + 1.96 * se)

print("naive_type1_ci=", mc_ci(naive_type1, n_sims))
print("gs_type1_ci=", mc_ci(gs_type1, n_sims))
